## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Hello There In this notebokk we are going to scrape the Full time and part time jobs data for the data related jobs like:<br>
Data analyst, ML dev, Data scientist, Data engineer etc ..<br><br>

**Important note:**
We will analyze some countries only for indeed becuase indeed needs to specify which country to look at.

## <center><strong>Importing <span style= 'color: #48e0dc'>Packeges

In [2]:
import time
import random
import pickle
import sqlite3
import warnings
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import count
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from IPython.display import set_matplotlib_formats
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [3]:
%matplotlib inline
COLORS = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']
MAX_PAGES = 300

set_matplotlib_formats('pdf', 'svg')
warnings.filterwarnings('ignore')

ip_port_list = [('75.84.199.80', '80'),
                ('172.173.241.207', '80'),
                ('209.145.60.213', '80'),
                ('86.109.3.28', '80'),
                ('65.111.241.211', '80'),
                ('144.34.162.125', '80')]

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_19340\1277823779.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= 'color: #5cd3f7'> web scrapers

In [4]:
data_jobs_titles = ['Data entry', 'Data engineer',
                    'Data scientist', 'Data analyst',
                    'ML devoloper']

indeed_countries = ['de', 'uk']#, 'www'] USA don't have to be spesfied in the URL so we will use 'www' instead
linkedin_countries = ['European Union', 'United States']
ip_port = random.choice(ip_port_list)

In [12]:
def scrape_page(url: str, retrieve_new_url= False) -> BeautifulSoup:
    
    driver = webdriver.Firefox()
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    
    # Wait until the page is fully loaded
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    html = driver.page_source

    if retrieve_new_url:
        
        new_url = driver.current_url
        driver.quit()
        
        return BeautifulSoup(html), new_url
        
    driver.quit()

    return BeautifulSoup(html)

In [13]:
# def indeed_scraper(page: int,
#                    country: str,
#                    job_title: str) -> BeautifulSoup:

#     modefid_page = (page - 1) * 10

#     data_job_title = job_title.replace(' ', '+')

#     # Pages in indeed.com are zero based indexed
#     url: str = f'https://{country}.indeed.com/jobs?q={data_job_title}&sc=0kf%3Aattr%28DSQF7%29%3B&start={modefid_page}' + \
#     '&pp=gQClAAAAAAAAAAAAAAACCXklWwCSAQIBK7YBBwDUbujiRGBahYW5TppjdUz7DjXn3aPZSbT47IoJ5LLbuzpYcXwZdzJ6rKHf' + \
#     '6gPWFkXVGxKKGxW-JAKb8BFo_hZAkBd7trBBTY32J2CrOuA3V9dGD_bre-lArmi9DRYlcah6hvoRfsYUNYSoQwIa8VOMZMxvH-s2Dlh' + \
#     'UPvUP-_Dz9ls4i-OLqVGpGh4AAA&vjk=b997bb0dddadea'

#     soup = scrape_page(url)
#     loaded_page = int(soup.find('button', 'css-ns2mzi e8ju0x51').text)

#     # available_pages = []

#     # for button in soup.find_all('button', 'css-1qt7hdn e8ju0x50'):
#     #     available_pages.append(int(button.text))

#     if (int(page) != loaded_page):
#         # We will use KeyError as standard to represent getting out of the max pages.
#         raise KeyError
        
#     print(f'Page {page} Loaded successfully.')

#     return  soup

In [37]:
def check_for_content(driver):

    global more_content_button_available
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

    try:
        driver.find_element(By.CSS_SELECTOR, 'loader loader--show')
        more_content_button_available = False
        continue_ = True

    except Exception as e:
        print(e)
        try:
            driver.find_element(By.CSS_SELECTOR, 'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible')
            more_content_button_available = True
            continue_ = True

        except:
            continue_ = False
            more_content_button_available = False

    return  continue_

In [38]:
def linkedin_scraper(country: str, # This scraper scrapes all pages at once
                     job_title: str) -> BeautifulSoup:

    # I know that code logic is wrong but at least it's running completly fine.

    driver = webdriver.Chrome()
    url: str = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&' + \
                f'location={country}&refresh=true&start=0&position=1&pageNum=0'

    driver.get(url)

    while True:
        try:
            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: check_for_content(driver))

            if more_content_button_available:
                print('Hello')
                more_content_button = driver.find_element(By.CLASS_NAME,
                                    'infinite-scroller__show-more-button infinite-scroller__show-more-button--visible')
                more_content_button.click()

        except:
            break

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    return  soup

## <center><strong>Collecting<span style= 'color: #5cd3f7'> jobs </span> data

**Now we will need for three steps to collect the data we need:**

1. Collect all pages we need from each scraper and stack them together as HTML code.
2. Scrape for jobs cards links from linkedin and indeed.
3. Scrape for the data in those links and store the data in a DataFrame then in SQLite
4. We may also consider collecting total results per each platform, country etc ..

#### **Stack pages together as HTML code**

In [39]:
soups = {'soup': [],
         'job_title': [],
         'country': [],
         'platform': []}

flag: bool = False
max_page: int = 7

# Indeed scraping TODO
# for country in indeed_countries:
#     for job_title in data_jobs_titles:
#         stacked_pages_soup: str = ''

#         for i in count(0):

#             try:
#                 page = i + 1
#                 soup: BeautifulSoup = indeed_scraper(country= country,
#                                                      job_title= job_title,
#                                                      page= page)
                
#                 stacked_pages_soup += '\n<br> ' + str(soup)
#                 if page > max_page:
#                     print('Finished loading current country or job title.\n')
#                     break

#             except KeyError as e:
#                 print('Finished loading current country or job title.\n')
#                 break

#         soups['platform'].append('Indeed')
#         soups['soup'].append(stacked_pages_soup)
#         soups['job_title'].append(job_title)
#         soups['country'].append(country)

# Linked In
print('Starting LinkedIn scraper.')
for country in linkedin_countries:

    print(f'Loading the {country} data.')
    for job_title in data_jobs_titles:


        soup: BeautifulSoup = linkedin_scraper(country= country,
                                job_title= job_title)


        # soups['platform'].append('LinkedIn')
        # soups['soup'].append(soup)
        # soups['job_title'].append(job_title)
        # soups['country'].append(country)

        print(f'Finished loading {job_title} jobs from the {country}.')
    print('\n')

Starting LinkedIn scraper.
Loading the European Union data.
Message: no such element: Unable to locate element: {"method":"css selector","selector":"loader loader--show"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x007BA813+48355]
	(No symbol) [0x0074C4B1]
	(No symbol) [0x00655358]
	(No symbol) [0x006809A5]
	(No symbol) [0x00680B3B]
	(No symbol) [0x006AE232]
	(No symbol) [0x0069A784]
	(No symbol) [0x006AC922]
	(No symbol) [0x0069A536]
	(No symbol) [0x006782DC]
	(No symbol) [0x006793DD]
	GetHandleVerifier [0x00A1AABD+2539405]
	GetHandleVerifier [0x00A5A78F+2800735]
	GetHandleVerifier [0x00A5456C+2775612]
	GetHandleVerifier [0x008451E0+616112]
	(No symbol) [0x00755F8C]
	(No symbol) [0x00752328]
	(No symbol) [0x0075240B]
	(No symbol) [0x00744FF7]
	BaseThreadInitThunk [0x762700C9+25]
	RtlGetAppContainerNam

Exception ignored in: <function Service.__del__ at 0x00000235D3BE7790>
Traceback (most recent call last):
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\site-packages\selenium\webdriver\common\service.py", line 181, in __del__
    self.stop()
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\site-packages\selenium\webdriver\common\service.py", line 142, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\site-packages\selenium\webdriver\common\service.py", line 121, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "C:\Users\FreeComp\.conda\envs\data_jobs_analysis\lib\urllib\request.py", l

Finished loading Data entry jobs from the European Union.
Message: no such element: Unable to locate element: {"method":"css selector","selector":"loader loader--show"}
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x007BA813+48355]
	(No symbol) [0x0074C4B1]
	(No symbol) [0x00655358]
	(No symbol) [0x006809A5]
	(No symbol) [0x00680B3B]
	(No symbol) [0x006AE232]
	(No symbol) [0x0069A784]
	(No symbol) [0x006AC922]
	(No symbol) [0x0069A536]
	(No symbol) [0x006782DC]
	(No symbol) [0x006793DD]
	GetHandleVerifier [0x00A1AABD+2539405]
	GetHandleVerifier [0x00A5A78F+2800735]
	GetHandleVerifier [0x00A5456C+2775612]
	GetHandleVerifier [0x008451E0+616112]
	(No symbol) [0x00755F8C]
	(No symbol) [0x00752328]
	(No symbol) [0x0075240B]
	(No symbol) [0x00744FF7]
	BaseThreadInitThunk [0x762700C9+25]
	RtlGetAppContainerNamed

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(soups).head(10)

#### **Collecting jobs links**

In [63]:
soups = pd.DataFrame(soups)

soups['total_jobs'] = np.nan
soups['jobs_links'] = np.nan
soups['companies_links'] = np.nan


# Collecting indeed jobs links TOFIX
# for i, soup in enumerate(soups[soups['platform'] == 'Indeed']['soup']):
    
#     indeed_jobs_links: list = []
#     indeed_companies_links: list = []

#     for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
#         job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
#         indeed_jobs_links.append(job_link)

#     for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
#         company_link = BeautifulSoup(scrape_page(job_link)).find('a', 'css-775knl emf9s7v0')
#         indeed_companies_links.append(company_link)
#         print(company_link)
        
#     soups['total_jobs'][i] = len(indeed_jobs_links)
#     soups['jobs_links'][i] = indeed_jobs_links
#     soups['companies_links'][i] = indeed_companies_links


# Collecting linkedIn jobs links
for i, soup in enumerate(soups[soups['platform'] == 'LinkedIn']['soup']):

    url_tempelate = 'https://www.linkedin.com###'
    linkedin_jobs_links: list = []
    linkedin_companies_links: list = []

    for job in soup.find_all('a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'):
        job_link = job['href']
        linkedin_jobs_links.append(job_link)

    # for job_link in linkedin_jobs_links[:5]: # TOREMOVE
    #     company_link = url_tempelate.replace('###', scrape_page(job_link).find(
    #         'img', 'artdeco-entity-image artdeco-entity-image--square-5 lazy-loaded')) #<- ['href']
    #     # print(company_link)
    #     linkedin_companies_links.append(company_link)

    soups['total_jobs'][i] = len(linkedin_jobs_links)
    soups['jobs_links'][i] = linkedin_jobs_links
    # soups['companies_links'][i] = linkedin_companies_links

In [65]:
scrape_page(linkedin_jobs_links[0]).find_all(href=True)
# .find('img', 'artdeco-entity-image artdeco-entity-image--square-5 lazy-loaded')['data-ghost-url']

[<link href="https://be.linkedin.com/jobs/view/data-entry-entry-level-online-typist-remote-at-admired-services-3661594237" rel="canonical"/>,
 <link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>,
 <link href="https://static.licdn.com/aero-v1/sc/h/248euz6dnkrrxp1xycd4fx00z" rel="stylesheet"/>,
 <a class="skip-link btn-md btn-primary absolute z-11 -top-[100vh] focus:top-0" href="#main-content">
       Skip to main content
     </a>,
 <a class="nav__logo-link link-no-visited-state z-1 mr-auto babybear:z-0 babybear:mr-0 babybear:flex-1 hover:no-underline focus:no-underline active:no-underline" data-tracking-control-name="public_jobs_nav-header-logo" data-tracking-will-navigate="" href="/?trk=public_jobs_nav-header-logo">
 <span class="sr-only">LinkedIn</span>
 <icon class="nav-logo--inbug flex text-color-brand papabear:hidden mamabear:hidden" data-svg-class-name="h-[34px] w-[34px] babybear:h-[27px] babybear:w-[27px]"></icon>
 <icon class="block text-co

In [ ]:
print(test_soup)

In [39]:
len(soups['soup'][1].find_all('a', 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'))

148

In [137]:
indeed_jobs_lin

('https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=297a5f03a72d28ba',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=dfb22eace90ab68a',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=1df95ac216434def',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=ec52d6d183156757',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=4be75a2883ca49b4',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=ce840e854164d5f5',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=7a31018777b2a0fb',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0ce08416199b1cb1',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=148d931cb5870ea9',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=88849361571a22a8',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=68040c6377ffaaf5',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=722a2feac0caa4db',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=db7619ac3e3a8b6f',
 'https://ww

In [84]:
for i, soup in enumerate(soups[soups['platform'] == 'Indeed']['soup']):

    indeed_jobs_links: set = set()
    indeed_companies_links: set = set()

    for job in soup.find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
        job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', BeautifulSoup((job)['data-jk'])
        indeed_jobs_links.add(job_link)

    for job_link in indeed_jobs_links[:5]: # WE WILL REMOVE THIS LIMIT
        company_link = scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href']
        indeed_companies_links.add(company_link)

    soups['total_jobs'][i] = len(indeed_jobs_links)
    soups['jobs_links'][i] = indeed_jobs_links
    soups['companies_links'][i] = indeed_companies_links

{'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=0f4e49c98c0536f1',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=19f3a3be9f35218f',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=1df95ac216434def',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=297a5f03a72d28ba',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=40e35f9a902792ed',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=446ba27dfffdbe36',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=48746eece7f6d42f',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=4be75a2883ca49b4',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=5010e97afafcfbf9',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=605f5275b7921be9',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=973410652b64f9a8',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=a34fc6b420419272',
 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=bbae1d266770b45a',
 'https://ww

In [117]:
for soup in soups[soups['platform'] == 'Indeed']['soup'][:2]:
    job_links = []

    for job in BeautifulSoup(soup).find_all('a', 'jcs-JobTitle css-jspxzf eu4oa1w0'):
        job_link = 'https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=###'.replace('###', job['data-jk'])
        job_links.append(job_link)

     for job_link in job_links[:5]:
         print(scrape_page(job_link).find('a', 'css-775knl emf9s7v0')['href'])

    print('\n')

27
120
120
120


In [115]:
len(set(job_links))

387

In [114]:
len(job_links) 

387

In [ ]:
test_html = scrape_page('https://www.indeed.com/jobs?q=d&l=&from=searchOnHP&vjk=26c4d3754f443acb')


In [ ]:
test_html.find_all('div','jobsearch-JobMetadataFooter')